In [1]:
import time
import os
import json
import csv
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.by import By

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict


In [9]:
def input_keyword(keyword):
    # (str) -> str
    # return url which includes keyword
    
    # url = 'https://www.indeed.com/?from=gnav-homepage'
    url = "https://www.indeed.com/jobs?q=&l=" + keyword + "&start="
    return url
    

In [53]:
def move_page(url, keyword):
    '''
    (str) -> ()
    
    '''
    page = input("총 몇 페이지까지 크롤링할까요? ")
    
    for p in range(0, int(page), 10):
        print(url+str(p))
        get_info(url+str(p), keyword)
        print(p, '페이지 크롤링 완료')
        print()

In [77]:
def get_info(url, keyword):
    options = webdriver.ChromeOptions()	
    # options.add_argument('headless')	
    options.add_argument('disable-gpu')	
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    driver.get(url)
    # 가끔 '구독'하라는 '팝업창이 뜸'
    try:
        corp_list = driver.find_elements_by_tag_name("div.jobsearch-SerpJobCard")
        link_list= driver.find_elements_by_tag_name('h2.title > a')
    except:
        driver.find_element_by_xpath("//*[@id='popover-close-link']").click()
        print("팝업... BYEBYE")
        time.sleep(5)
        corp_list = driver.find_elements_by_tag_name("div.jobsearch-SerpJobCard")
        link_list= driver.find_elements_by_tag_name('h2.title > a')
    
    print(corp_list)
    new_driver = webdriver.Chrome('chromedriver.exe', options=options)
    for idx, corp in enumerate(corp_list):
        time.sleep(2)
        print(corp)
        # Message: stale element reference: element is not attached to the page document
        #   (Session info: chrome=83.0.4103.116)
        # 해결법: https://stackoverflow.com/questions/18225997/stale-element-reference-element-is-not-attached-to-the-page-document
        # 동일한 행위 한 번 더 
        try:
            time.sleep(2)
            detail_link = link_list[idx].get_attribute('href')
            print(detail_link)
        except Exception as e:
            time.sleep(2)
            print(e)
            detail_link = link_list[idx].get_attribute('href')
            print(detail_link)
        
        
        time.sleep(5)
        new_driver.get(detail_link)
        
        corp_cnt = list()

        try:
            job_title = corp.find_element_by_tag_name("h2.title").text.split('\n')[0]
        except:
            job_title = "NULL"
        try:
            company_name = corp.find_element_by_tag_name("span.company").text
        except:
            company_name = "NULL"
        try:
            location = corp.find_element_by_tag_name("div.sjcl > span").text
        except:
            location = "NULL"
        try:
            minus_date = corp.find_element_by_tag_name("span.date").text.split(" ")[0]
            try:
                minus_date = int(minus_date)
                minus_date = str(date.today() + relativedelta(days=-minus_date))
            except:
                if minus_date == 'Today' or 'Just':
                    minus_date = str(date.today())
                else:
                    minus_date = 'at least a month ago'
        except:
            minus_date = "NULL"
        try:
            short_descript = corp.find_element_by_tag_name("div.summary").text
        except:
            short_descript = "NULL"
            
        time.sleep(3)
        try:
            detail = new_driver.find_element_by_tag_xpath('/html/body/div[1]/div[1]/div[3]/div/div/div[1]/div[1]').text
            print('1번', detail[:50])
        except:
            try:
                detail = new_driver.find_element_by_xpath("//*[@id='jobDescriptionText']").text
                print('2번', detail[:50])
            except Exception as e:
                detail = "NULL"
                print('3번', e)
        
        corp_cnt.append(job_title)
        corp_cnt.append(company_name)
        corp_cnt.append(location)
        corp_cnt.append(minus_date)
        corp_cnt.append(short_descript)
        corp_cnt.append(detail)
        corp_cnt.append(detail_link)
        
        driver.back()
        time.sleep(3)
        write_csv(keyword, "_indeed_ver3", corp_cnt)
    driver.close()

In [78]:
#    wr.writrow(['job_title', 'company_name', 'location', 'minus_date', 'short_descript'])
def write_csv(keyword, filename, info_list):
    csv_file = open(str(keyword)+str(filename)+'.csv', "a", encoding = 'utf-8-sig', newline= '')
    wr = csv.writer(csv_file)
    wr.writerow(info_list)
    csv_file.close()

In [79]:
def main():
    keyword = input("검색 키워드를 입력하세요")
    url = input_keyword(keyword)
    move_page(url, keyword)

In [ ]:
if __name__ == "__main__":
    main()